# Extracció emojis

In [8]:
import tweepy
import math
from datetime import datetime, timedelta, timezone
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import time
import json
import regex
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import emoji
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from textblob import TextBlob

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iairm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Funcions per a treballar amb emojis

In [ ]:
import emoji

#Funcio per treure els emojis d'un text.
def deEmojify(text):

    string = ""
    data = regex.findall(r'\X', text)
    for word in data:
        for char in word:
            if not char in emoji.UNICODE_EMOJI:
                string = string + char

    return string

#Funcio per obtenir un text sense emojis.
def Emojify(text):

    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

#Funcion que retorna un diccionari amb el count de cada emoji.
def countEmojis(text,mainDict):
    lt = list(Emojify(text)) # Set deleted
    if "🇪🇦" in lt:
        lt.remove("🇪🇦")
        lt.append("🇪🇸")
    if "❤" in lt:
        lt.remove("❤")
        lt.append("❤️")
    for emoji in lt:
        if emoji in mainDict:
            mainDict[emoji] += 1
        else:
            mainDict[emoji] = 1
    return mainDict

## Emojis count per a cada partit

In [13]:
import emoji

# Llegim els csv de cada partit politic.
pp = pd.read_csv("pp.csv")
psoe = pd.read_csv("psoe.csv")
cs = pd.read_csv("cs.csv")
vox = pd.read_csv("vox.csv")
up = pd.read_csv("up.csv")

finaldu = [] # Llista amb les ocurrencies d'emojis per a cada partit.

userDict = {} # Diccionari on guardarem les ocurrencies de cada emoji de tots els usuaris del partit actual.
#Vox Users
voxUsers = vox[["user_name","user_screen_name","user_description","user_id"]].groupby(by=["user_id"],axis=0).first()

for i, user in voxUsers.iterrows(): 
    userDict = countEmojis(user["user_name"],userDict)
    if not pd.isna(user["user_description"]):
        userDict = countEmojis(user["user_description"],userDict)
      

    
duVox = {k: v for k, v in sorted(userDict.items(), reverse=True, key=lambda item: item[1])}
duVox = list(duVox.items())
finaldu.append(duVox)

userDict = {} # Diccionari on guardarem les ocurrencies de cada emoji de tots els usuaris del partit actual.
#Pp Users
ppUsers = pp[["user_name","user_screen_name","user_description","user_id"]].groupby(by=["user_id"],axis=0).first()

for i, user in ppUsers.iterrows(): 
    userDict = countEmojis(user["user_name"],userDict)
    if not pd.isna(user["user_description"]):
        userDict = countEmojis(user["user_description"],userDict)
duPP = {k: v for k, v in sorted(userDict.items(), reverse=True, key=lambda item: item[1])}
duPP = list(duPP.items())
finaldu.append(duPP)

userDict = {} # Diccionari on guardarem les ocurrencies de cada emoji de tots els usuaris del partit actual.
#Ciu Users
csUsers = cs[["user_name","user_screen_name","user_description","user_id"]].groupby(by=["user_id"],axis=0).first()

for i, user in csUsers.iterrows(): 
    userDict = countEmojis(user["user_name"],userDict)
    if not pd.isna(user["user_description"]):
        userDict = countEmojis(user["user_description"],userDict)
        
duCs = {k: v for k, v in sorted(userDict.items(), reverse=True, key=lambda item: item[1])}
duCs = list(duCs.items())
finaldu.append(duCs)

userDict = {} # Diccionari on guardarem les ocurrencies de cada emoji de tots els usuaris del partit actual.
#Psoe Users
psoeUsers = psoe[["user_name","user_screen_name","user_description","user_id"]].groupby(by=["user_id"],axis=0).first()

for i, user in psoeUsers.iterrows(): 
    userDict = countEmojis(user["user_name"],userDict)
    if not pd.isna(user["user_description"]):
        userDict = countEmojis(user["user_description"],userDict)
        
duPsoe = {k: v for k, v in sorted(userDict.items(), reverse=True, key=lambda item: item[1])}
duPsoe = list(duPsoe.items())
finaldu.append(duPsoe)

userDict = {} 
#Podemos Users
upUsers = up[["user_name","user_screen_name","user_description","user_id"]].groupby(by=["user_id"],axis=0).first()

for i, user in upUsers.iterrows(): 
    userDict = countEmojis(user["user_name"],userDict)
    if not pd.isna(user["user_description"]):
        userDict = countEmojis(user["user_description"],userDict)
        
duUP = {k: v for k, v in sorted(userDict.items(), reverse=True, key=lambda item: item[1])}
duUP = list(duUP.items())
finaldu.append(duUP)

In [14]:
endList = []
transactions = []
for dt in finaldu:
    List = []
    transaction = []
    for i,value in dt:
        if value >= 4:
            List.append((i,value))
            transaction.append(i)
    endList.append(List)
    transactions.append(transaction)

print(endList) #Llista amb els emojis y ocurrencies per a cada partit.


flatTrans = []
for sublist in transactions:
    for item in sublist:
        flatTrans.append(item)

#Conjunt dels emojis diferents que apareixen (tenint en compte tots els partits politics). Util per quan calculem el tfidf.
diffEmojis = set(flatTrans)

[[('🇪🇸', 64), ('💚', 23), ('🖤', 10)], [('🇪🇸', 23), ('💙', 16), ('🅿️', 8)], [('🍊', 44), ('🇪🇸', 43), ('🇪🇺', 22), ('🖤', 15), ('🧡', 11), ('🏳️\u200d🌈', 7), ('❤️', 4)], [('🌹', 52), ('❤️', 34), ('✊', 6), ('🇪🇸', 6), ('🇪🇺', 4), ('🔻', 4), ('💜', 4)], [('💜', 59), ('🔻', 25), ('✊', 16), ('❤️', 14), ('💛', 14)]]


## Proves per a calcular score d'un tweet

In [16]:
"""
#Utilitzarem el tfidf.
parties = ["VOX", "PP", "CS", "PSOE", "UP"]

DannyTw = "Danny 🇪🇸💜🇪💛🇸🇪🇸"

emojiIdf = {}
for emoji in diffEmojis:
    emojiIdf[emoji] = 0
    for index, party in enumerate(endList):
        for partyEemoji, num in party:
            if partyEemoji == emoji:
                emojiIdf[emoji] += 1
                
print(emojiIdf)

import emoji
DannyDict = countEmojis(DannyTw, {})
DannyList = list(DannyDict.items())
DannyList

voxEmojisNum = np.sum([value for i, value in endList[0]])
ppEmojisNum = np.sum([value for i, value in endList[1]])
csEmojisNum = np.sum([value for i, value in endList[2]])
psoeEmojisNum = np.sum([value for i, value in endList[3]])
upEmojisNum = np.sum([value for i, value in endList[4]])
EmojisPartiesSum = [voxEmojisNum, ppEmojisNum, csEmojisNum, psoeEmojisNum, upEmojisNum]

scores = {}
for index, party in enumerate(endList):
    scores[index] = 0
    
for i, value in DannyList:
    for index, party in enumerate(endList):
        for emoji, num in party:
            if i == emoji:
                scores[index] += value * np.log(len(endList)/emojiIdf[emoji]) * num/EmojisPartiesSum[index]

print(scores)
list(scores.values())/np.sum(list(scores.values()))
"""

{'🧡': 1, '🍊': 1, '💙': 1, '💛': 1, '🔻': 2, '🇪🇸': 4, '💜': 2, '✊': 2, '💚': 1, '🇪🇺': 2, '🅿️': 1, '🏳️\u200d🌈': 1, '🌹': 1, '🖤': 2, '❤️': 3}
{0: 0.14722873488772603, 1: 0.10919790808993243, 2: 0.06572036100350014, 3: 0.04549112941256254, 4: 0.5983850308957231}


array([0.15240704, 0.1130386 , 0.06803187, 0.04709114, 0.61943135])